In [15]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector

In [16]:
import math

In [17]:
detector = HandDetector(detectionCon=0.8, maxHands=1)

In [18]:
class SnakeGameClass:
    def __init__(self):
        self.points = [] # All point of the snake
        self.lengths = [] # Distance between each points
        self.currentLength = 0 # Total Length of the sanke
        self.allowedLength = 150 # Total allowed length
        self.previousHead = 0,0 # Previous head point
    
    def update(self, imgMain, currentHead):
        px, py = self.previousHead
        cx, cy = currentHead
        
        self.points.append([cx, cy])
        distance = math.hypot(cx-px, cy-py)
        self.lengths.append(distance)
        self.currentLength += distance
        self.previousHead = cx, cy
    
        # Draw snake
        for i, point in enumerate(self.points):
            if i != 0:
                cv2.line(imgMain, self.points[i-1], self.points[i], (0, 0, 255), 20)
        cv2.circle(imgMain, self.points[-1], 20, (200,0,200), cv2.FILLED)
        
        return imgMain

In [19]:
game = SnakeGameClass()

In [20]:
cap = cv2.VideoCapture(1)
cap.set(3, 1280)
cap.set(4, 720)

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)
    
    if hands:
        lmList = hands[0]['lmList']
        pointIndex = lmList[8][0:2]
        img = game.update(img, pointIndex)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    cv2.imshow('Image', img)
    cv2.waitKey(1)
    
cap.release()
cv2.destroyAllWindows()